In [9]:
import pandas as pd
import numpy as np
import csv


# Strategy: Loop through data set of games and then for each game loop through data set of plays
# Use plays data set to add points and possesions to counter for each player


#### Define function to keep track of players, points for/against, and possessions:

class Player:
    def __init__(self, team, id):
        self.team = team
        self.id = id
        self.pointsFor = 0
        self.pointsAgainst = 0
        self.offPos = 0
        self.defPos = 0

    def offensivePointsAndPossession(self, points):
        self.pointsFor += points
        self.offPos += 1

    def defensivePointsAndPossession(self, points):
        self.pointsAgainst += points
        self.defPos += 1

    def offensivePoints(self, points):
        self.pointsFor += points

    def defensivePoints(self, points):
        self.pointsAgainst += points

    def offensivePossession(self):
        self.offPos += 1

    def defensivePossession(self):
        self.defPos += 1


def pointsAndPossession(points):
    for player in floor:
        if player.team == play["Team_id"]:
            player.offensivePointsAndPossession(points)
        else:
            player.defensivePointsAndPossession(points)

def pointsOnly(points):
    for player in floor:
        if player.team == play["Team_id"]:
            player.offensivePoints(points)
        else:
            player.defensivePoints(points)

def possessionOnly(offensiveTeam):
    for player in floor:
        if player.team == offensiveTeam:
            player.offensivePossession()
        else:
            player.defensivePossession()

In [19]:
### Creating csv file for output, reading in txt files,
### and creating lists for total roster and on floor lineups:

# Creates a new .csv file and open for writing.
answer = open("try.csv", "w")
output = csv.writer(answer)

# Writes the column headers in the answer file in .csv format
answerHeaders = ["Game_ID", "Player_ID", "OffRtg", "DefRtg"]
output.writerow(answerHeaders)

# Opens the file with all the plays, and turns it into a data frame using pandas.
playFile = pd.read_csv("Play_by_Play.txt", delimiter="\t")

# Opens the file with the lineups, and turns it into a data frame using pandas.
# We then group the data based on their Game_id, and the period.
lineupFile = pd.read_csv("Game_Lineup.txt", delimiter="\t")
lineup = lineupFile.groupby(["Game_id","Period"])

# These are the two lists needed to keep track of the roster of everyone in the game,
# as well as everyone on the floor.
roster = []
floor = []

In [20]:
##### Eh needs work
### Event Grouping

# These two lists will be used for free throw actions. 20 types of free throws,
# but can split all of them into two distinct groups based on
# whether they are reboundable or not.
FTActionsNotReboundable = [11,13,14,16,17,18,19,20,21,22,25,26,27,28,29]
FTActionsReboundable = [10,12,15]
# Unsure what 16 and 17 mean????


FTHold = [3,4,7,8,9,11,14,15,18,20]

actionPlays = [1,2,3,5,12,13]

# This is a boolean variable, used in the occasional case when on a rebound play,
# we are not told who secured the rebound.
# When this happens, whoever is the team of the next action following, that becomes the team that
# secured the previous plays rebound. #When this is set to "True", verify the team.
# It must originally be set to "false".
needToCheck = False

# Boolean for when a foul is made and there is potential for a substitution.
# The list will hold the substitutions that must be made after the final free throws are complete.
subFreeze = False
heldSubs = []

In [21]:
##### ooof

### Ordering the data, filling roster and floor list for each game
### For-loops:

## This is the main outer loop grouping plays by game:

for game,group in playFile.groupby("Game_id"):
    
    # Sort the game on the basis of the world clock time first, and then on event number.
    # This would give the true ordering for the game we will loop through.
    sortedGames = group.sort_values(by=["Period","PC_Time","WC_Time","Event_Num"],
                                       ascending=[True,False,True,True])
    
    # Reset the index for each games then remove it
    sortedGames = sortedGames.reset_index(drop=True)

    ## Whenever a new game begins, we fill a new roster list, as well as a new floor list.
    
    # For roster list:
    # Use the lineup data and grab group that has the correct Game_id (game) and has a period of zero (0).
    GameRosterList = lineup.get_group((game,0))
    
    # Creates list for every player on each team at the start of each game
    roster = [Player(player["Team_id"],player["Person_id"])
              for index,player in GameRosterList.iterrows()]
    
    # For on-floor (starters) list:
    # Use lineup data and grab group of the same game (game), but from first quarter (1).
    StartersList = lineup.get_group((game, 1))
    
    # Create list for game starters
    floor = [athlete for athlete in roster if any(athlete.id == starter["Person_id"]
                                                  for index,starter in StartersList.iterrows())]
    
    ### Nested-loop for each play of game:
    
    # For-loop going through each play, adding possesions/points to each player and keeping track of subs:
    for index,play in sortedGames.iterrows():
        
        # If the Event_Msg_Type is equal to "12", and the period is greater than "1",
        # then this is the start of a period that is not the start of a game.
        # Here we must just change the floor lineup in case of a substitution during the quarter break.
        if play["Event_Msg_Type"] == 12 and play["Period"] > 1:
        
            # Exact same logic as filling floor list at the beginning of a game.
            LineUpNewQuarter = lineup.get_group((game, play["Period"]))
            floor = [athlete for athlete in roster if any(athlete.id == starter["Person_id"] 
                                                          for index,starter in LineUpNewQuarter.iterrows())]
            
        # If the Event_Msg_Type is a "1", then it's a made FG,
        # which is the end of a possession. For every player on the floor,
        # depending on their team, call the function to add to their offensive or defensive stats
        # and add to possession count.
        if play["Event_Msg_Type"] == 1:
            pointsAndPossession(play["Option1"])
            
            # Keep tracking time
            shotTime = play["PC_Time"]
            playClock = play["PC_Time"]

        # If the FG has been missed, we set the variable "play["Team_id"]"
        # to the team that took the shot. Since every missed shot is followed
        # by a rebound, we must check on the rebound to see if it was an offensive or defensive rebound.
        
        #### Ehhhhh not really sure right here homies:
        elif play["Event_Msg_Type"] == 2:
            reboundTime = sortedGames.loc[index+1]["PC_Time"]

            rebounder = None
            for candidate in floor:
                if candidate.id == sortedGames.loc[index+1]["Person1"]:
                    rebounder = candidate
                    break

            if rebounder is None:
                if sortedGames.loc[index+2]["Team_id"] != play["Team_id"]:
                    foulClock = sortedGameData.loc[index+1]["PC_Time"]
                    possessionOnly(play["Team_id"])


            # If the team of the rebounder is different then "play["Team_id"]",
            # then the opposing team to the shooting team got the rebound.
            # This results in a possession change, but since no points were scored,
            # we only add the appropriate offensive or defensive possessions to the players on the floor.
            # If the rebounder's team is the same as "play["Team_id"]", then this was an offensive rebound,
            # so we can ignore it since no possession changes, and no points were scored.
            elif rebounder.team != play["Team_id"]:
                possessionOnly(play["Team_id"])
                if heldSubs:
                    for spot, bench in heldSubs:
                        if bench.team == play["Team_id"]:
                            bench.offPos -= 1
                        else:
                            bench.defPos -= 1

        # An Event_Msg_Type of "3" is a free throw which has a few different types.
        elif play["Event_Msg_Type"] == 3:
            
         ### Go through free throw types (this part is a bitch)

SyntaxError: unexpected EOF while parsing (<ipython-input-21-25b000e20465>, line 98)